### 3.charseq 
> hihello의 단어사전 버전에 불과, 

> 단일(Window이동X) many-to-many임

> 단일 Layer(FC X)

In [2]:
import torch
import torch.optim as optim
import numpy as np

# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

#### 단어사전 만들기

In [16]:
sample = " if you want you"

# make dictionary
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'i': 0, 'w': 1, 'y': 2, 't': 3, 'o': 4, ' ': 5, 'n': 6, 'a': 7, 'u': 8, 'f': 9}


In [17]:
dic_size = len(char_dic)
dic_size # input_size 가 된다

10

In [18]:
# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print(X.shape) #  if you want yo (공백부터 시작)
print(Y.shape) # if you want you

torch.Size([1, 15, 10])
torch.Size([1, 15])


- ( Batch = 1 )
- 전체 row (Window 이동 횟수) = 1  ※ 윈도우 이동 X
- Sequence(Windiow) : 15 
- input_size(특징차원, 단어사전/원핫, input node) : 10
- output_size : 15 (to-many 이므로 sequence 길이와 같음)

In [19]:
X

tensor([[[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]])

In [22]:
Y

tensor([[0, 9, 5, 2, 4, 8, 5, 1, 7, 6, 3, 5, 2, 4, 8]])

In [23]:
# hyper parameters
input_size = dic_size         # 10
hidden_size = dic_size  # 10 
learning_rate = 0.1

# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

#### SIZE 확인 위해 출력

In [25]:
# start training
for i in range(50):
    optimizer.zero_grad()
    
    outputs, _status = rnn(X)
    print("output: ", outputs.shape)
    print("_status: ", _status.shape)
    
    # 3D → 2D 로 변환 후 비교
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))   # Outputs : [1, 15, 10] → [15,10],   
                                                                                                      #              Y : [1,15]        → [15]
    print("outputs.view(-1, input_size) 변형결과 : ", outputs.view(-1, input_size).shape)
    print("Y.view(-1) 변형결과 : ",Y.shape, "→", Y.view(-1).shape, '\n')    
    
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

output:  torch.Size([1, 15, 10])
_status:  torch.Size([1, 1, 10])
outputs.view(-1, input_size) 변형결과 :  torch.Size([15, 10])
Y.view(-1) 변형결과 :  torch.Size([1, 15]) → torch.Size([15]) 

0 loss:  0.8192166686058044 prediction:  [[0 9 5 2 4 8 5 1 7 6 3 5 2 4 8]] true Y:  [[0, 9, 5, 2, 4, 8, 5, 1, 7, 6, 3, 5, 2, 4, 8]] prediction str:  if you want you
output:  torch.Size([1, 15, 10])
_status:  torch.Size([1, 1, 10])
outputs.view(-1, input_size) 변형결과 :  torch.Size([15, 10])
Y.view(-1) 변형결과 :  torch.Size([1, 15]) → torch.Size([15]) 

1 loss:  0.8188547492027283 prediction:  [[0 9 5 2 4 8 5 1 7 6 3 5 2 4 8]] true Y:  [[0, 9, 5, 2, 4, 8, 5, 1, 7, 6, 3, 5, 2, 4, 8]] prediction str:  if you want you
output:  torch.Size([1, 15, 10])
_status:  torch.Size([1, 1, 10])
outputs.view(-1, input_size) 변형결과 :  torch.Size([15, 10])
Y.view(-1) 변형결과 :  torch.Size([1, 15]) → torch.Size([15]) 

2 loss:  0.818506121635437 prediction:  [[0 9 5 2 4 8 5 1 7 6 3 5 2 4 8]] true Y:  [[0, 9, 5, 2, 4, 8, 5, 1, 7, 6, 3, 

### ' if you want yo' → 'if you want you' 로 잘 예측하였음

### Clean 소스 버전

In [14]:
# # start training
# for i in range(50):
#     optimizer.zero_grad()
#     outputs, _status = rnn(X)
#     loss = criterion(outputs.view(-1, dic_size), Y.view(-1))   # Outputs : [1, 15, 10] → [15,10],   
#                                                                                                       #              Y : [1,15]        → [15]
#     loss.backward()
#     optimizer.step()

#     result = outputs.data.numpy().argmax(axis=2)
#     result_str = ''.join([char_set[c] for c in np.squeeze(result)])
#     print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)